# Asymmetry Detection Models
This notebook shows the process of load and train of a LSTM neuronal network to classify the asymmetry laterality of a PD patient or a control. In this notebook
we will show different implementations of LSTMs (Normal and bidirectional) and with different hyperparameters.

## Requiered Imports

In [1]:
import pickle
import datetime

import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import TensorBoard


from src.model.asa_models import build_basic_lstm, build_2layer_lstm

from src.settings import ROOT_DIR
from src.utils.data_split import get_features_target
from src.model.callbacks_builder import get_callbacks, build_tensorboard

C:\Users\julia\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (5.2.0)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


## Data load

In [2]:
train_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'train.pkl'
val_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'val.pkl'

with open(train_data_path, 'rb') as file:
    train_data = pickle.load(file)

with open(val_data_path, 'rb') as file:
    val_data = pickle.load(file)

In [3]:
len(train_data)

124

In [4]:
X_train, y_train = get_features_target(train_data)
X_val, y_val = get_features_target(val_data)

In [5]:
max_length = max(len(x) for x in X_train)
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post', dtype='float32')

max_length = max(len(x) for x in X_val)
X_val_padded = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen=max_length, padding='post', dtype='float32')

## Tensor Board
This tool will help us to supervise the experimentation process of all the experimental models described in this notebook

In [6]:
%load_ext tensorboard
%tensorboard --logdir ../results/asa/lightning_logs

Reusing TensorBoard on port 6006 (pid 17168), started 3 days, 13:51:19 ago. (Use '!kill 17168' to kill it.)

In [7]:
tensorboard_callback = build_tensorboard('asa')

## Models

### Simple LSTM
This model is a 64-lstm with any hidden layers and just one dense layer to infer the output.
Optimizer: *Adam*
Loss: *binary_crossentropy*

In [8]:
BATCH_SIZE = 14
EPOCHS = 200

In [9]:
model = build_basic_lstm()
checkpoint_model, early_stop_model = get_callbacks('asa', '64-lstm-no-hidden')

In [10]:
X_train_padded

array([[[-5.43115005e+01,  1.59815016e+01, -8.23789215e+00, ...,
         -3.35979372e-01, -3.89058217e-02,  1.92625320e-03],
        [ 2.08804154e+00,  5.41658401e+01,  7.23621597e+01, ...,
         -1.80081815e-01, -1.11523336e-02, -8.65482446e-03],
        [ 1.24695072e+01,  7.35638657e+01,  9.12297058e+01, ...,
          9.48639885e-02,  2.56922767e-02, -1.41362306e-02],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-2.09597816e+01,  4.49966393e+01,  5.01408310e+01, ...,
          1.81523055e-01,  1.80250814e-03, -1.05616525e-02],
        [-7.06980286e+01,  5.30379143e+01,  4.13741264e+01, ...,
          6.68321773e-02,  7.75659355e

In [11]:
# Train the model
history_model = model.fit(
    X_train_padded, 
    np.array(y_train), 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    callbacks=[tensorboard_callback, checkpoint_model, early_stop_model])

Epoch 1/200
4/9 [============>.................] - ETA: 31s - loss: 0.6925 - accuracy: 0.6071

KeyboardInterrupt: 

### 2 Hidden layers LSTM

In [ ]:
BATCH_SIZE = 14
EPOCHS = 150

In [0]:
lstm = build_2layer_lstm()
checkpoint_lstm, early_stop_lstm = get_callbacks('asa', 'lstm-2hidden')

In [0]:
lstm.summary()

In [0]:
history_lstm = lstm.fit(x=X_train,
                        y=y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        callbacks=[tensorboard_callback, checkpoint_lstm, early_stop_lstm])